<center>
    <p style="text-align:center">
        <img alt="arize langchain pinecone logos" src="https://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/langchain-pinecone-search-and-retrieval/arize_langchain_pinecone.png" width="750"/>
        <br>
        <a href="https://cloud.google.com/vertex-ai">VertexAI</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">Arize-Phoenix</a>
        |
        <a href="https://github.com/hwchase17/langchain">LangChain</a>
        |
        <a href="https://www.pinecone.io/">Pinecone</a>
    </p>
</center>
<h1 align="center">Evaluating and Improving Search and Retrieval Applications</h1>

Imagine you're an engineer at Arize AI and you've built and deployed a documentation question-answering service using LangChain, Pinecone, and Google's VertexAI platform. Users send questions about Arize's core product via a chat interface, and your service retrieves chunks of your indexed documentation in order to generate a response to the user. As the engineer in charge of maintaining this system, you want to evaluate the quality of the responses from your service.

Phoenix helps you:
- identify gaps in your documentation
- detect queries for which the LLM gave bad responses
- detect failures to retrieve relevant context

In this tutorial, you will:

- Ask questions of a LangChain application backed by VertexAI and Pinecone over a knowledge base of the Arize documentation
- Use Phoenix to visualize user queries and knowledge base documents to identify areas of user interest not answered by your documentation
- Find clusters of responses with negative user feedback
- Identify failed retrievals using query density, cosine similarity, Euclidean distance, and LLM-assisted ranking metrics

Parts of this notebook require a GCP account with the VertexAI API enabled and a Pinecone API key to run. If you don't have these, you can still use Phoenix by skipping cells with the ❗ emoji.

⚠️ This notebook assumes you've already indexed the Arize documentation. Run this [script](https://github.com/Arize-ai/phoenix/blob/vertexai-search-and-retrieval/tutorials/build_arize_docs_index_langchain_pinecone.py) to build your own Pinecone index over the Arize documentation with VertexAI.

Let's get started!

## 1. Install Dependencies and Import Libraries

Install Phoenix, LangChain, Pinecone, and the VertexAI API.

In [ ]:
!pip install -q arize-phoenix google-cloud-aiplatform langchain pinecone-client

Import libraries.

In [ ]:
import textwrap
from typing import Dict, List, Optional, Tuple

import numpy as np
import numpy.typing as npt
import pandas as pd
import phoenix as px
import pinecone
import vertexai
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatVertexAI
from langchain.docstore.document import Document
from langchain.embeddings.vertexai import VertexAIEmbeddings
from langchain.vectorstores import Pinecone
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
from tqdm import tqdm
import vertexai
from vertexai.preview.language_models import TextGenerationModel

pd.set_option("display.max_colwidth", None)

## 2. Configure Your GCP Project and Location

❗ Configure your GCP project and location.

In [ ]:
gcp_project_id = "copy paste your gcp project id here"
gcp_location = "copy paste your gcp location here"
assert gcp_project_id != "copy paste your gcp project id here", "❌ Please set your GCP project id"
assert gcp_location != "copy paste your gcp location here", "❌ Please set your GCP location"
vertexai.init(project=gcp_project_id, location=gcp_location)

## 3. Configure Your Pinecone API Key and Environment

❗ Configure your Pinecone API key and environment.

In [ ]:
pinecone_api_key = "copy paste your api key here"
pinecone_environment = "copy paste your environment here"
pinecone_index_name = "copy paste your index name here"
assert pinecone_api_key != "copy paste your api key here", "❌ Please set your Pinecone API key"
assert (
    pinecone_environment != "copy paste your environment here"
), "❌ Please set your Pinecone environment"
assert (
    pinecone_index_name != "copy paste your index name here"
), "❌ Please set your Pinecone index name"
pinecone.init(api_key=pinecone_api_key, environment=pinecone_environment)

## 4. Run Your Question-Answering Service

❗ Define a wrapper class around `lanchain.embeddings.vertexai.VertexAIEmbeddings` to record query and document embeddings.

In [ ]:
class VertexAIEmbeddingsWrapper(VertexAIEmbeddings):
    """
    Wrapper around VertexAIEmbeddings that stores the query and document embeddings in memory.
    """

    query_text_to_embedding: Dict[str, List[float]] = {}
    document_text_to_embedding: Dict[str, List[float]] = {}

    def embed_query(self, text: str) -> List[float]:
        embedding = super().embed_query(text)
        self.query_text_to_embedding[text] = embedding
        return embedding

    def embed_documents(self, texts: List[str], batch_size: int = 5) -> List[List[float]]:
        embeddings = super().embed_documents(texts, batch_size)
        for text, embedding in zip(texts, embeddings):
            self.document_text_to_embedding[text] = embedding
        return embeddings

    @property
    def query_embedding_dataframe(self) -> pd.DataFrame:
        return self._convert_text_to_embedding_map_to_dataframe(self.query_text_to_embedding)

    @property
    def document_embedding_dataframe(self) -> pd.DataFrame:
        return self._convert_text_to_embedding_map_to_dataframe(self.document_text_to_embedding)

    @staticmethod
    def _convert_text_to_embedding_map_to_dataframe(
        text_to_embedding: Dict[str, List[float]]
    ) -> pd.DataFrame:
        texts, embeddings = map(list, zip(*text_to_embedding.items()))
        embedding_arrays = [np.array(embedding) for embedding in embeddings]
        return pd.DataFrame.from_dict(
            {
                "text": texts,
                "text_vector": embedding_arrays,
            }
        )

❗ Define a wrapper around `langchain.vectorstores.Pinecone` to record retrieval data, including the retrieved pieces of context for each query and the associated similarity scores between query and context embeddings.

In [ ]:
class PineconeWrapper(Pinecone):
    query_text_to_document_score_tuples: Dict[str, List[Tuple[Document, float]]] = {}

    def similarity_search_with_score(
        self,
        query: str,
        k: int = 4,
        filter: Optional[dict] = None,
        namespace: Optional[str] = None,
    ) -> List[Tuple[Document, float]]:
        document_score_tuples = super().similarity_search_with_score(
            query=query,
            k=k,
            filter=filter,
            namespace=namespace,
        )
        self.query_text_to_document_score_tuples[query] = document_score_tuples
        return document_score_tuples

    @property
    def retrieval_dataframe(self) -> pd.DataFrame:
        query_texts = []
        document_texts = []
        retrieval_ranks = []
        scores = []
        for query_text, document_score_tuples in self.query_text_to_document_score_tuples.items():
            for retrieval_rank, (document, score) in enumerate(document_score_tuples):
                query_texts.append(query_text)
                document_texts.append(document.page_content)
                retrieval_ranks.append(retrieval_rank)
                scores.append(score)
        return pd.DataFrame.from_dict(
            {
                "query_text": query_texts,
                "document_text": document_texts,
                "retrieval_rank": retrieval_ranks,
                "score": scores,
            }
        )

❗ Define an instance of `langchain.chains.RetrievalQA` using `VertexAIEmbeddingsWrapper` and `PineconeWrapper`.

In [ ]:
num_retrieved_documents = 2
embeddings = VertexAIEmbeddingsWrapper()
docsearch = PineconeWrapper.from_existing_index(
    index_name=pinecone_index_name,
    embedding=embeddings,
)
llm = ChatVertexAI()
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": num_retrieved_documents}),
)

❗ Run a query and display the results, including the final response from the chatbot, the retrieved pieces of context and associated similarity scores, and a portion of the query embedding.

In [ ]:
query_text = "How do I get an Arize API key?"
# query_text = "What's the difference between primary and baseline datasets?"
# query_text = "How do I send in extra metadata with each record?"
# query_text = "How does Arize's surrogate explainability model work?"

response_text = chain.run(query_text)
retrievals_df = docsearch.retrieval_dataframe.tail(num_retrieved_documents)
contexts = retrievals_df["document_text"].to_list()
scores = retrievals_df["score"].to_list()
query_embedding = embeddings.query_embedding_dataframe["text_vector"].iloc[-1]

print("Response")
print("========")
print()
for line in textwrap.wrap(response_text.strip(), width=80):
    print(line)
print()

for context_index, (context, score) in enumerate(zip(contexts, scores)):
    print(f"Retrieved Context {context_index}")
    print("===================")
    print()

    for line in textwrap.wrap(context.strip(), width=80):
        print(line)
    print()
    print(f"score: {score}")
    print()

print("Query Embedding")
print("===============")
print()

print(query_embedding)
print()
print(f"dimension: {len(query_embedding)}")

## 5. Load Your Data Into Pandas Dataframes

To use Phoenix, you must load your data into Pandas dataframes. First, load your knowledge base into a dataframe.

In [ ]:
database_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/langchain-pinecone/database.parquet"
)
database_df.head()

The columns of your dataframe are:
- **text:** the chunked text in your knowledge base
- **text_vector:** the embedding vector for the text, computed during the LangChain build using the "textembedding-gecko" embedding model from VertexAI

Next, download a dataframe containing query data.

In [ ]:
query_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/langchain-pinecone/langchain_pinecone_query_dataframe_with_user_feedbackv2.parquet"
)
query_df.head()

The columns of the dataframe are:
- **text:** the query text
- **text_vector:** the embedding representation of the query, captured from LangChain at query time
- **response:** the final response from the LangChain application
- **context_text_0:** the first retrieved context from the knowledge base
- **context_similarity_0:** the cosine similarity between the query and the first retrieved context
- **context_text_1:** the second retrieved context from the knowledge base
- **context_similarity_1:** the cosine similarity between the query and the first retrieved context
- **user_feedback:** approval or rejection from the user (-1 means thumbs down, +1 means thumbs up)

The query and database datasets are drawn from different distributions; the queries are short questions while the database entries are several sentences to a paragraph. The embeddings from VertexAI's "textembedding-gecko" capture these differences and naturally separate the query and context embeddings into distinct regions of the embedding space. When using Phoenix, you want to "overlay" the query and context embedding distributions so that queries appear close to their retrieved context in the Phoenix point cloud. To achieve this, we compute a centroid for each dataset that represents an average point in the embedding distribution and center the two distributions so they overlap.

In [ ]:
database_centroid = database_df["text_vector"].mean()
database_df["centered_text_vector"] = database_df["text_vector"].apply(
    lambda x: x - database_centroid
)
query_centroid = query_df["text_vector"].mean()
query_df["centered_text_vector"] = query_df["text_vector"].apply(lambda x: x - query_centroid)

## 6. Compute Proxy Metrics for Retrieval Quality

Cosine similarity and Euclidean distance can act as proxies for retrieval quality. The cosine distance between query and retrieved context was computed at query time and is part of the query dataframe downloaded above. Compute the Euclidean distance between each query embedding and retrieved context embedding and add corresponding columns to the query dataframe.

In [ ]:
def compute_euclidean_distance(
    vector0: npt.NDArray[np.float32], vector1: npt.NDArray[np.float32]
) -> float:
    """
    Computes the Euclidean distance between two vectors.
    """
    return np.linalg.norm(vector0 - vector1)


num_retrieved_documents = 2
for context_index in range(num_retrieved_documents):
    euclidean_distances = []
    for _, row in query_df.iterrows():
        query_embedding = row["text_vector"]
        context_text = row[f"context_text_{context_index}"]
        database_row = database_df[database_df["text"] == context_text].iloc[0]
        database_embedding = database_row["text_vector"]
        euclidean_distance = compute_euclidean_distance(query_embedding, database_embedding)
        euclidean_distances.append(euclidean_distance)
    query_df[f"euclidean_distance_{context_index}"] = euclidean_distances
query_df.head()

## 7. Run LLM-Assisted Evaluations

Cosine similarity and Euclidean distance are reasonable proxies for retrieval quality, but they don't always work perfectly. A novel idea is to use LLMs to measure retrieval quality by simply asking the LLM whether each piece of retrieved context is relevant or irrelevant to the corresponding query.

Use Google's `text-bison` model to predict whether each retrieved document is relevant or irrelevant to the query.

In [ ]:
EVALUATION_PROMPT_TEMPLATE = """You will be given a query and a reference text. You must determine whether the reference text contains an answer to the input query. Your response must be binary (0 or 1) and should not contain any text or characters aside from 0 or 1. 0 means that the reference text does not contain an answer to the query. 1 means the reference text contains an answer to the query.

# Query: {query}

# Reference: {reference}

# Binary: """


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def evaluate_query_and_retrieved_context(
    query: str,
    context: str,
    model,
) -> str:
    evaluation_prompt = EVALUATION_PROMPT_TEMPLATE.format(
        query=query,
        reference=context,
    )
    response = model.predict(
        evaluation_prompt,
        temperature=0.1,
        max_output_tokens=5,
    )
    return response.text


def evaluate_retrievals(
    retrievals_data: Dict[str, str],
    model_name: str,
) -> List[str]:
    model = TextGenerationModel.from_pretrained(model_name)
    responses = []
    for query, retrieved_context in tqdm(retrievals_data.items()):
        response = evaluate_query_and_retrieved_context(
            query,
            retrieved_context,
            model,
        )
        responses.append(response)
    return responses


def process_binary_responses(
    binary_responses: List[str], binary_to_string_map: Dict[int, str]
) -> List[str]:
    """
    Parse binary responses and convert to the desired format
    converts them to the desired format. The binary_to_string_map parameter
    should be a dictionary mapping binary values (0 or 1) to the desired
    string values (e.g. "irrelevant" or "relevant").
    """
    processed_responses = []
    for binary_response in binary_responses:
        try:
            binary_value = int(binary_response.strip())
            processed_response = binary_to_string_map[binary_value]
        except (ValueError, KeyError):
            processed_response = None
        processed_responses.append(processed_response)
    return processed_responses

sample_query_df = query_df.head(10).copy()
evaluation_model_name = "text-bison"
for context_index in range(num_retrieved_documents):
    retrievals_data = {
        row["text"]: row[f"context_text_{context_index}"] for _, row in sample_query_df.iterrows()
    }
    raw_responses = evaluate_retrievals(retrievals_data, evaluation_model_name)
    processed_responses = process_binary_responses(raw_responses, {0: "irrelevant", 1: "relevant"})
    sample_query_df[f"text_bison_relevance_{context_index}"] = processed_responses
sample_query_df[
    ["text", "context_text_0", "text_bison_relevance_0", "context_text_1", "text_bison_relevance_1"]
].head(10)

Running evaluations across the entire dataset takes a while, so download a dataset of pre-computed evaluations and add to the query dataframe.

In [ ]:
text_bison_evaluations_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/langchain-pinecone/langchain_pinecone_openai_evaluations.parquet",
)[["text", "text_bison_relevance_0", "text_bison_relevance_1"]]
query_df = pd.merge(query_df, text_bison_evaluations_df, on="text")
query_df[["text", "context_text_0", "context_text_1", "text_bison_relevance_0", "text_bison_relevance_1"]]

## 8. Compute Ranking Metrics

Now that you know whether each piece of retrieved context is relevant or irrelevant to the corresponding query, you can compute precision@k for k = 1, 2 for each query. This metric tells you what percentage of the retrieved context is relevant to the corresponding query.

precision@k = (# of top-k retrieved documents that are relevant) / (k retrieved documents)

If your precision@2 is greater than zero for a particular query, your LangChain application successfully retrieved at least one relevant piece of context with which to answer the query. If the precision@k is zero for a particular query, that means that no relevant piece of context was retrieved.

Compute precision@k for k = 1, 2 and view the results.

In [ ]:
num_relevant_documents_array = np.zeros(len(query_df))
for retrieved_context_index in range(0, num_retrieved_documents):
    num_retrieved_documents = retrieved_context_index + 1
    num_relevant_documents_array += (
        query_df[f"text_bison_relevance_{retrieved_context_index}"]
        .map(lambda x: int(x == "relevant"))
        .to_numpy()
    )
    query_df[f"text_bison_precision_at_{num_retrieved_documents}"] = pd.Series(
        num_relevant_documents_array / num_retrieved_documents
    )

query_df[
    [
        "text_bison_relevance_0",
        "text_bison_relevance_1",
        "text_bison_precision_at_1",
        "text_bison_precision_at_2",
    ]
]

## 9. Launch Phoenix

Define a schema to tell Phoenix what the columns of your query and database dataframes represent (features, predictions, actuals, tags, embeddings, etc.). See the [docs](https://docs.arize.com/phoenix/) for guides on how to define your own schema and API reference on `phoenix.Schema` and `phoenix.EmbeddingColumnNames`.

In [ ]:
query_df["response_vector"] = query_df[
    "centered_text_vector"
].copy()  # the response requires an embedding, but we don't have one, so we just use the prompt embedding
query_schema = px.Schema(
    prompt_column_names=px.EmbeddingColumnNames(
        raw_data_column_name="text",
        vector_column_name="centered_text_vector",
    ),
    response_column_names=px.EmbeddingColumnNames(
        raw_data_column_name="response",
        vector_column_name="response_vector",
    ),
    tag_column_names=[
        "context_text_0",
        "context_similarity_0",
        "context_text_1",
        "context_similarity_1",
        "euclidean_distance_0",
        "euclidean_distance_1",
        "text_bison_relevance_0",
        "text_bison_relevance_1",
        "text_bison_precision_at_1",
        "text_bison_precision_at_2",
        "user_feedback",
    ],
)
database_schema = px.Schema(
    prompt_column_names=px.EmbeddingColumnNames(
        raw_data_column_name="text",
        vector_column_name="centered_text_vector",
    ),
)

Create Phoenix datasets that wrap your dataframes with the schemas that describe them.

In [ ]:
database_ds = px.Dataset(
    dataframe=database_df,
    schema=database_schema,
    name="database",
)
query_ds = px.Dataset(
    dataframe=query_df,
    schema=query_schema,
    name="query",
)

Launch Phoenix. Follow the instructions in the cell output to open the Phoenix UI.

In [ ]:
session = px.launch_app(query_ds, database_ds)